<a href="https://colab.research.google.com/github/steelannelida/nanoGPT/blob/master/nanogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/steelannelida/nanoGPT.git

Cloning into 'nanoGPT'...
remote: Enumerating objects: 688, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 688 (delta 2), reused 0 (delta 0), pack-reused 682 (from 1)
Receiving objects: 100% (688/688), 958.82 KiB | 2.91 MiB/s, done.
Resolving deltas: 100% (387/387), done.


In [2]:
!pip install torch numpy transformers datasets tiktoken wandb tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00


In [3]:
!yes  | python nanoGPT/data/shakespeare/prepare.py

train has 301,966 tokens
val has 36,059 tokens


In [4]:
with open('nanoGPT/data/shakespeare/input.txt') as f:
  text = f.read()

chars = sorted(set(text))
vocab_size=len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}

def encode(t):
  return [stoi[c] for c in t]

def decode(seq):
  return ''.join([chars[i] for i in seq])

decode(encode("sandwitch"))

'sandwitch'

In [5]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data = torch.tensor(encode(text), device=device)
data.shape, data.dtype

n = int(data.shape[0] * 0.9)
train_data = data[:n]
valid_data = data[n:]

In [6]:
#torch.manual_seed(1337)


def get_batch(data_set=train_data, batch_size=32, seq_length=128):
    x = torch.zeros([batch_size, seq_length], dtype=torch.int, device=device)
    y = torch.zeros([batch_size, seq_length], dtype=torch.int, device=device)
    for b in range(batch_size):
        t = torch.randint(0, data_set.shape[0] - seq_length - 1, [1])
        x[b] = data_set[t:t+seq_length]
        y[b] = data_set[t+1:t+seq_length+1]
    return x, y

x, y = get_batch()
print(decode(x[13]))
print(decode(y[13]))

:
Either you must
Confess yourselves wondrous malicious,
Or be accused of folly. I shall tell you
A pretty tale: it may be you h

Either you must
Confess yourselves wondrous malicious,
Or be accused of folly. I shall tell you
A pretty tale: it may be you ha


In [13]:
x.shape, x.

RuntimeError: expand(torch.cuda.IntTensor{[32, 128]}, size=[0]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)

In [32]:
import torch.nn as nn

class LM(nn.Module):
  def __init__(self, vocab_size, embed_size=256, nheads=16, max_pos=2048):
    super().__init__()
    self.embeddings = nn.Embedding(vocab_size, embed_size)
    self.pos_embeddings = nn.Embedding(max_pos, embed_size)
    self.attn = nn.MultiheadAttention(embed_size, nheads, batch_first=True)
    self.out = nn.Linear(embed_size, vocab_size)
    self.max_pos = max_pos
    self.to(device)

  def forward(self, idx):
    idx = torch.tensor(idx, device=device)
    l = idx.shape[-1]
    pe = self.pos_embeddings(torch.arange(0, l, device=device))
    e = self.embeddings(idx)
    e = e + pe.view(1, *pe.shape)
    mask = torch.tril(torch.ones([l, l], device=device)).T
    #print(mask)
    a, w = self.attn.forward(e, e, e, attn_mask=mask)
    logits = self.out.forward(a)
    return logits

  def generate(self, prompt, l):
    prompt = torch.tensor(prompt)
    pl = prompt.shape[0]
    result = torch.zeros([pl + l], dtype=torch.int, device=device)
    result[:pl] = prompt
    for i in range(l):
      logits = self.forward(result[:pl + i])
      sm = logits[:,-1].flatten().softmax(0)
      next_idx = torch.multinomial(sm, 1)
      result[i+pl] = next_idx
    return result

model = LM(vocab_size)

k=x
q=x[:,:-1]
model.forward(encode('jello'))
decode(model.generate(encode('hello'), 10))

<ipython-input-32-4fe09e28ffee>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx = torch.tensor(idx, device=device)


'helloHRr-uts-I&'

In [33]:
x, y = get_batch()
logits = model(x)
loss_fun = nn.CrossEntropyLoss()
loss = loss_fun(logits.permute(0, 2, 1), y.long())
loss

<ipython-input-32-4fe09e28ffee>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx = torch.tensor(idx, device=device)


tensor(4.1815, device='cuda:0', grad_fn=<NllLoss2DBackward0>)

In [34]:
torch.manual_seed(1337)

model = LM(vocab_size)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
xv, yv = get_batch(valid_data, batch_size=4)


In [38]:


for step in range(2000):
  x, y = get_batch(batch_size=64)
  model.train()
  logits = model.forward(x)
  loss = loss_fun(logits.permute(0, 2, 1), y.long())
  opt.zero_grad()
  loss.backward()
  opt.step()

  if step % 100 == 0:
    model.eval()
    vlogits = model.forward(xv)
    vloss = loss_fun(vlogits.permute(0,2,1), yv.long())
    print('%d\t%f\t%f'%(step, loss, vloss))


<ipython-input-32-4fe09e28ffee>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx = torch.tensor(idx, device=device)


0	0.019898	0.019464
100	0.018559	0.018453
200	0.018284	0.018816
300	0.019890	0.021130
400	0.017624	0.019966
500	0.018642	0.019403
600	0.016219	0.020911
700	0.019056	0.020845
800	0.016032	0.016454
900	0.016456	0.015693
1000	0.015709	0.015612
1100	0.017914	0.019018
1200	0.016581	0.021275
1300	0.017575	0.021961
1400	0.019625	0.019714
1500	0.017829	0.016779
1600	0.017545	0.018821
1700	0.017664	0.018570
1800	0.016717	0.016393
1900	0.015633	0.016869


In [37]:
print(decode(x[0]))


FRIAR LAURENCE:
God pardon sin! wast thou with Rosaline?

ROMEO:
With Rosaline, my ghostly father? no;
I have forgot that name,


In [40]:
print(decode(model.generate(encode('ROMEO:'), 120)))

ROMEO:NEOMOOOOOMOOOOOOMOOOOOOOOOOOOOOOOOOOOOOOOOOOMOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO


<ipython-input-32-4fe09e28ffee>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx = torch.tensor(idx, device=device)
